目前用了一个小数据，保存为data_syn_with_label_20.csv，里面有“text”评价信息和“label”标签信息，“label”仅分为“正面”和“负面”2类。现在需要做以下操作：1.bert进行词向量表征 ，label 需要怎么处理 ； 2.创建数据集； 3.将数据集加载到简单的cnn模型中进行训练，测试是否可用；

In [2]:
# 1. 加载数据
# 首先，加载CSV文件，该文件包含文本评价信息和相应的标签。
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# 加载数据
data = pd.read_csv("data_syn_with_label_20.csv")
print("Loaded data:", data.head())

Loaded data:                                                 text label
0  五点 十多 去 店里 都 没开 灯 很 暗 真的 很 影响 用餐 体验 南昌 吃 感觉 完全...    负面
1  朋友 带来 吃 说 霸占 南昌 平价 火锅 悠闲 味道 确实 不赖 四个 女生 加 三次 菜...    正面
2                            美味 服务 好 经常 带 小孩 一家人 去 吃    正面
3  季季红 火锅 食材 新鲜 味道 正宗 锅底 浓郁 调味 恰到好处 服务周到 热情 火锅 爱好...    正面
4  非常 美味 真的 推荐 吃 都 美味 服务 好 态度 已经 不是 一次 两次 吃 超 喜欢 ...    正面


In [3]:
# 2. 处理标签
# 对于二分类任务，使用 LabelEncoder 将文本标签转换为整数是合适的。这个过程将 "正面" 和 "负面" 分别映射为 0 和 1。
# 初始化并应用LabelEncoder
label_encoder = LabelEncoder()
data['label_encoded'] = label_encoder.fit_transform(data['label'])

# 显示转换后的标签以确保正确
print("Label Encoding:", data[['label', 'label_encoded']].drop_duplicates())


Label Encoding:   label  label_encoded
0    负面              1
1    正面              0


In [4]:
import pandas as pd
from transformers import BertTokenizer, TFBertModel
from sklearn.preprocessing import LabelEncoder
import numpy as np
import tensorflow as tf

# 加载数据
data = pd.read_csv("data_syn_with_label_20.csv")

# 初始化BERT tokenizer和模型
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model_bert = TFBertModel.from_pretrained('bert-base-uncased')

# Tokenize 文本
inputs = tokenizer(data['text'].tolist(), max_length=128, truncation=True, padding='max_length', return_tensors="tf")

# 使用BERT模型生成词向量
outputs = model_bert(inputs['input_ids'], attention_mask=inputs['attention_mask'])
bert_features = outputs.last_hidden_state

# 使用平均池化获取每个句子的向量表示
pooled_bert_features = tf.reduce_mean(bert_features, axis=1)

# 处理标签
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(data['label'])


c:\Python312\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [5]:
# 创建TensorFlow数据集
def create_dataset(features, labels):
    dataset = tf.data.Dataset.from_tensor_slices((features, labels))
    dataset = dataset.shuffle(len(labels)).batch(10)  # 适当调整batch size
    return dataset

# 转换features为适合CNN输入的形式
dataset = create_dataset(pooled_bert_features, labels)


In [7]:
# 加载至简单CNN模型进行测试是否可行
# 调整模型输入层以适应数据形状：如果您打算使用平均池化后的BERT输出（形状为 [None, 768]），
# 应该修改模型结构以接受这种形状的输入。由于数据已经是池化后的输出，您可能不需要一个复杂的CNN结构来处理它。一个简单的全连接层可能就足够了。
# 修正模型结构：基于BERT的输出形状，下面是一个调整后的模型示例。
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout

# 创建适应平均池化输出的模型
model = Sequential([
    Dense(128, activation='relu', input_shape=(768,)),  # 输入层现在直接接受平均池化的768维向量
    Dropout(0.2),
    Dense(1, activation='sigmoid')  # 用于二分类
])

model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 训练模型
model.fit(dataset, epochs=10)



Epoch 1/10


c:\Python312\Lib\site-packages\keras\src\layers\core\dense.py:86: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 5ms/step - accuracy: 0.4000 - loss: 0.7830  
Epoch 2/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9333 - loss: 0.2874
Epoch 3/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9000 - loss: 0.3357
Epoch 4/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9333 - loss: 0.2003
Epoch 5/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9333 - loss: 0.1898
Epoch 6/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9333 - loss: 0.2516
Epoch 7/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9667 - loss: 0.1462
Epoch 8/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9333 - loss: 0.2093
Epoch 9/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.9000 - loss: 0.1895
Epoch 10/10
2/2 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - accuracy: 0.9667 - loss: 0.1611


到这一步，说明20行的小数据实验成功运行，接下来，处理整个数据，带标签的约8千万数据 
先进行BERT词向量表征 
然后创建数据集
再次生成训练集和测试集